## Exports nodes and edges from tweets (Retweets, Mentions, or Replies) [JSON]
Exports nodes and edges from tweets (either from retweets or mentions) in json format that can be exported from SFM, and saves it in a file format compatible with various social network graph tools such as Gephi, Cytoscape, Kumu, etc. These are for directed graphs.

In [325]:
import sys
import json
import re
import numpy as np
from datetime import datetime
import pandas as pd  

tweetfile = '/home/soominpark/sfmproject/Work/Emma Briant case/elites_sample.json'

### 1. Export edges from Retweets, Mentions, or Replies
* Run one of three blocks of codes below for your purpose. 

In [28]:
# 1. Export edges from Retweets

fh = open(tweetfile, 'r')

userdata = pd.DataFrame(columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count' ))
edges = pd.DataFrame(columns=('Source','Target','Strength'))

for line in fh:
    try:
        tweet = json.loads(line)
    except:
        continue
    if 'retweeted_status' not in tweet:
        continue
    
    userdata = userdata.append(pd.DataFrame([[tweet['user']['id_str'],
                                tweet['user']['screen_name'],
                                tweet['user']['created_at'],
                                tweet['user']['profile_image_url_https'],
                                tweet['user']['followers_count'],
                                tweet['user']['friends_count']]], columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count')), ignore_index=True)
    userdata = userdata.append(pd.DataFrame([[tweet['retweeted_status']['user']['id_str'],
                                tweet['retweeted_status']['user']['screen_name'],
                                tweet['retweeted_status']['user']['created_at'],
                                tweet['retweeted_status']['user']['profile_image_url_https'],
                                tweet['retweeted_status']['user']['followers_count'],
                                tweet['retweeted_status']['user']['friends_count']]], columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count')), ignore_index=True)                 
    edges = edges.append(pd.DataFrame([[tweet['user']['id_str'],
                                tweet['retweeted_status']['user']['id_str'],
                                str(datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y'))]]
                                , columns=('Source','Target','Strength')), ignore_index=True)           

In [ ]:
# 2. Export edges from Mentions

fh = open(tweetfile, 'r')

userdata = pd.DataFrame(columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count' ))
edges = pd.DataFrame(columns=('Source','Target','Strength'))

for line in fh:
    try:
        tweet = json.loads(line)
    except:
        continue
    if len(tweet['entities']['user_mentions']) == 0:
        continue
    
    for mention in tweet['entities']['user_mentions']:
        userdata = userdata.append(pd.DataFrame([[tweet['user']['id_str'],
                                tweet['user']['screen_name'],
                                tweet['user']['created_at'],
                                tweet['user']['profile_image_url_https'],
                                tweet['user']['followers_count'],
                                tweet['user']['friends_count']]], columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count')), ignore_index=True)
        if len(userdata[userdata['Id'].str.contains(mention['id_str'])]) == 0:
            userdata = userdata.append(pd.DataFrame([[tweet['user']['id_str'],
                                tweet['user']['screen_name'],
                                np.nan,
                                np.nan,
                                np.nan,
                                np.nan]], columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count')), ignore_index=True)
        edges = edges.append(pd.DataFrame([[tweet['user']['id_str'],
                                    mention['id_str'],
                                    str(datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y'))]]
                                    , columns=('Source','Target','Strength')), ignore_index=True)  

In [ ]:
# 3. Export edges from Replies

fh = open(tweetfile, 'r')

userdata = pd.DataFrame(columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count' ))
edges = pd.DataFrame(columns=('Source','Target','Strength'))

for line in fh:
    try:
        tweet = json.loads(line)
    except:
        continue
    if tweet['in_reply_to_user_id_str'] is None:
        continue

    userdata = userdata.append(pd.DataFrame([[tweet['user']['id_str'],
                                tweet['user']['screen_name'],
                                tweet['user']['created_at'],
                                tweet['user']['profile_image_url_https'],
                                tweet['user']['followers_count'],
                                tweet['user']['friends_count']]], columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count')), ignore_index=True)
    if len(userdata[userdata['Id'].str.contains(tweet['in_reply_to_user_id_str'])]) == 0:
            userdata = userdata.append(pd.DataFrame([[tweet['in_reply_to_user_id_str'],
                                tweet['in_reply_to_screen_name'],
                                np.nan,
                                np.nan,
                                np.nan,
                                np.nan]], columns=('Id','Label','user_created_at','profile_image','followers_count','friends_count')), ignore_index=True)
    edges = edges.append(pd.DataFrame([[tweet['user']['id_str'],
                                tweet['in_reply_to_user_id_str'],
                                str(datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y'))]]
                                , columns=('Source','Target','Strength')), ignore_index=True)

In [373]:
userdata.shape

(7791, 6)

In [372]:
edges.count()

Source      3990
Target      3990
Strength    3990
dtype: int64

In [370]:
userdata.tail(5)

,Id,Label,user_created_at,profile_image,followers_count,friends_count
7786,516106696,mdyblz10,Tue Mar 06 02:34:17 +0000 2012,https://pbs.twimg.com/profile_images/320513643...,1304.0,2566.0
7787,516106696,mdyblz10,NaN,NaN,NaN,NaN
7788,736745064697888769,ViciTrue,Sun May 29 02:24:46 +0000 2016,https://pbs.twimg.com/profile_images/767371065...,6899.0,6948.0
7789,736745064697888769,ViciTrue,NaN,NaN,NaN,NaN
7790,19626004,heyitsmelo,Wed Jan 28 00:03:29 +0000 2009,https://pbs.twimg.com/profile_images/782778248...,362.0,1194.0


In [371]:
edges.tail(5)

,Source,Target,Strength
3985,277980792,734886138326847491,2016-09-09 18:43:37
3986,528002721,734886138326847491,2016-09-10 03:41:06
3987,20430247,347627434,2016-08-13 15:25:57
3988,516106696,26487169,2016-10-09 02:06:13
3989,736745064697888769,367078366,2016-09-10 17:15:36


### 2. Leave only the tweets whose strength level >= user specified level (directed)

In [343]:
strengthLevel = 2  # Network connection strength level: the number of times in total each of the tweeters responded to or mentioned the other.
                   # If you have 1 as the level, then all tweeters who mentioned or replied to another at least once will be displayed. But if you have 5, only those who have mentioned or responded to a particular tweeter at least 5 times will be displayed, which means that only the strongest bonds are shown.

edges2 = edges.groupby(['Source','Target'])['Strength'].count()
edges2 = edges2.reset_index()
edges2 = edges2[edges2['Strength'] >= strengthLevel]

### 3. Export nodes

In [353]:
# Export nodes from the edges and add node attributes for both Sources and Targets.
userdata = userdata.sort_values(['Id','followers_count'], ascending=[True, False])
userdata = userdata.drop_duplicates(['Id'], keep='first') 

ids = edges2['Source'].append(edges2['Target']).to_frame()
ids.columns = ['Id']
ids = ids.drop_duplicates()

nodes = pd.merge(ids, userdata, on='Id', how='left')

In [356]:
nodes.head(5)

,Id,Label,user_created_at,profile_image,followers_count,friends_count
0,1022143381,RRBoulden,Wed Dec 19 13:57:17 +0000 2012,https://abs.twimg.com/sticky/default_profile_i...,18.0,71.0
1,1051550413,susiesam1,Tue Jan 01 01:14:10 +0000 2013,https://pbs.twimg.com/profile_images/770450345...,9.0,35.0
2,109949937,TurraloonNS,Sat Jan 30 20:31:43 +0000 2010,https://pbs.twimg.com/profile_images/594973377...,112.0,77.0
3,118504245,lminato,Sun Feb 28 22:08:11 +0000 2010,https://pbs.twimg.com/profile_images/754514218...,72.0,179.0
4,124199238,DSF2020,Thu Mar 18 15:49:49 +0000 2010,https://pbs.twimg.com/profile_images/668427205...,2855.0,475.0


In [355]:
ids.shape

(164, 1)

In [348]:
edges2.shape

(125, 3)

### 4. Export nodes and edges to csv files

In [362]:
# change column names for Kumu import (Run this when using Kumu)
nodes.columns = ['Id', 'Label', 'Date', 'Image', 'followers_count', 'friends_count']
edges2.columns = ['From','To','Strength']

In [363]:
# Print nodes to check
nodes.head(3)

,Id,Label,Date,Image,followers_count,friends_count
0,1022143381,RRBoulden,Wed Dec 19 13:57:17 +0000 2012,https://abs.twimg.com/sticky/default_profile_i...,18.0,71.0
1,1051550413,susiesam1,Tue Jan 01 01:14:10 +0000 2013,https://pbs.twimg.com/profile_images/770450345...,9.0,35.0
2,109949937,TurraloonNS,Sat Jan 30 20:31:43 +0000 2010,https://pbs.twimg.com/profile_images/594973377...,112.0,77.0


In [364]:
# Print edges to check
edges2.head(3)

,From,To,Strength
22,1022143381,759251,2
38,1051550413,26487169,2
74,109949937,777879358451093504,2


In [366]:
# Export nodes and edges to csv files
nodes.to_csv('nodes.csv', encoding='utf-8', index=False)
edges2.to_csv('edges.csv', encoding='utf-8', index=False)